In [1]:
import pandas as pd
import sys
import glob
import pytorch_lightning as pl
import torch
import pickle
from datasets import Dataset, DatasetDict, load_from_disk, concatenate_datasets
from functools import partial
from transformers.models.llama.tokenization_llama import LlamaTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import sentencepiece as spm
import os
from multiprocessing import Pool
from tqdm import tqdm
from datetime import datetime
from pytorch_lightning import seed_everything
import random
import gc
import h5py
import numpy as np

/opt/rh/rh-python38/root/usr/local/lib64/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
2023-10-19 10:57:38.108608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = '/data/rozen/home/e0833634/lama/data/ppi_8000'

In [78]:
df = pd.read_csv(path+'/ppigpt_test_merged_MI0915_LTPHTP_8000AA_oct10_2023.csv')
df

,Unnamed: 0,UniProtID_1,UniProtID_2,symbol_1,symbol_2,seq_1,seq_2,ppi,ppi_len,cluster,cluster_size
0,4,Q8SYG2,O46106,CSN3_DROME,NOI_DROME,MGSALENYVNQVRTLSASGSYRELAEELPESLSLLARNWSILDNVL...,METLLEQQRRLHEERERLVKLMVDEHATKKPGEKERIHSEHRLKYL...,MGSALENYVNQVRTLSASGSYRELAEELPESLSLLARNWSILDNVL...,948,Q8SYG2,4
1,25,P38961,P38961,RRP8_YEAST,RRP8_YEAST,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,784,P38961,1
2,26,P38961,Q06142,RRP8_YEAST,IMB1_YEAST,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,MSTAEFAQLLENSILSPDQNIRLTSETQLKKLSNDNFLQFAGLSSQ...,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,1253,P38961,1
3,27,P38961,Q02821,RRP8_YEAST,IMA1_YEAST,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,MDNGTDSSTSKFVPEYRRTNFKNKGRFSADELRRRRDTQQVELRKA...,MALFNVEGWSIKTKTVAFDNKTNKSSKDKKKNNRKNGKLTREQKLK...,934,P38961,1
4,39,P38064,P12687,RM16_YEAST,RM02_YEAST,MFPYLTRMNLSIKMGGLTLKESSPNAFLNNTTIARRFKHEYAPRFK...,MWNPILLDTSSFSFQKHVSGVFLQVRNATKRAAGSRTSMKDSAGRR...,MFPYLTRMNLSIKMGGLTLKESSPNAFLNNTTIARRFKHEYAPRFK...,603,P38064,1
...,...,...,...,...,...,...,...,...,...,...,...
49343,480409,Q05655,Q9NR28,KPCD_HUMAN,DBLOH_HUMAN,MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQ...,MAALKSWLSRSVTSFFRYRQCLCVPVVANFKKRCFSELIRPWHKTV...,MAPFLRIAFNSYELGSLQAEDEANQPFCAVKMKEALSTERGKTLVQ...,915,Q9WTQ0,6
49344,480414,P00749,P05121,UROK_HUMAN,PAI1_HUMAN,MRALLARLLLCVLVVSDSKGSNELHQVPSNCDCLNGGTCVSNKYFS...,MQMSPALTCLVLGLALVFGEGSAVHHPPSYVAHLASDFGVRVFQQV...,MRALLARLLLCVLVVSDSKGSNELHQVPSNCDCLNGGTCVSNKYFS...,833,P06869,2
49345,480416,P02930,P75831,TOLC_ECOLI,MACB_ECOLI,MKKLLPILIGLSLSGFSSLSQAENLMQVYQQARLSNPELRKSAADR...,MTPLLELKDIRRSYPAGDEQVEVLKGISLDIYAGEMVAIVGASGSG...,MKKLLPILIGLSLSGFSSLSQAENLMQVYQQARLSNPELRKSAADR...,1141,P02930,1
49346,480419,P26231,P14923,CTNA1_MOUSE,PLAK_HUMAN,MTAVHAGNINFKWDPKSLEIRTLAVERLLEPLVTQVTTLVNTNSKG...,MEVMNLMEQPIKVTEWQQTYTYDSGIHSGANTCVPSVSSKGIMEED...,MTAVHAGNINFKWDPKSLEIRTLAVERLLEPLVTQVTTLVNTNSKG...,1651,P26232,5


In [79]:
import sentencepiece as spm
tokenizer_path = '/data/rozen/home/e0833634/lama/protllama/batch_script/'
tokenizer = spm.SentencePieceProcessor(model_file=tokenizer_path+"protein_8k.model")

In [80]:
tokenizer.pad_token_id = tokenizer.unk_id()

In [16]:
tokenizer.encode('METLLEDYLHHFI<s>MET')

[523, 33, 897, 3046, 5765, 0, 5607]

In [85]:
seq1 = df['seq_1'].values.tolist()
len(seq1)

49348

In [86]:
seq2 = df['seq_2'].values.tolist()
len(seq1)

49348

In [87]:
df2 = pd.read_csv(path+'/ppigpt_train_merged_MI0915_LTPHTP_8000AA_oct10_2023.csv')
df2

,Unnamed: 0,UniProtID_1,UniProtID_2,symbol_1,symbol_2,seq_1,seq_2,ppi,ppi_len,cluster,cluster_size
0,0,P67809,Q13242,YBOX1_HUMAN,SRSF9_HUMAN,MSSEAETQQPPAAPPAAPALSAADTKPGTTGSGAGSGGPGGLTSAA...,MSGWADERGGEGDGRIYVGNLPTDVREKDLEDLFYKYGRIREIELK...,MSSEAETQQPPAAPPAAPALSAADTKPGTTGSGAGSGGPGGLTSAA...,545,P16989,6
1,1,Q8NER1,Q86SS6,TRPV1_HUMAN,SYT9_HUMAN,MKKWSSTDLGAAADPLQKDTCPDPLDGDPNSRPPPAKPQLSTAKSR...,MPGARDALCHQALQLLAELCARGALEHDSCQDFIYHLRDRARPRLR...,MKKWSSTDLGAAADPLQKDTCPDPLDGDPNSRPPPAKPQLSTAKSR...,1330,Q8NER1,2
2,2,O75319,Q13242,DUS11_HUMAN,SRSF9_HUMAN,MRNSETLERGVGGCRVFSCLGSYPGIEGAGLALLADLALGGRLLGT...,MSGWADERGGEGDGRIYVGNLPTDVREKDLEDLFYKYGRIREIELK...,MRNSETLERGVGGCRVFSCLGSYPGIEGAGLALLADLALGGRLLGT...,598,O75319,1
3,3,Q9W3N6,Q8SYG2,USO1_DROME,CSN3_DROME,MEFLKSGIKTVLGSTEPGQQPSAAETVEKLVDRVYSSTLLEDRRDA...,MGSALENYVNQVRTLSASGSYRELAEELPESLSLLARNWSILDNVL...,MEFLKSGIKTVLGSTEPGQQPSAAETVEKLVDRVYSSTLLEDRRDA...,1281,O60763,4
4,5,Q9W0S8,Q9VKD6,WAGC_DROME,DGT2_DROME,MQNLKIQEEVNSLMRLGQHFDDQLKLASVELGDFSDDDLALLDKCA...,MDDPSATLLPEHSEDLRLARDAELKKVLKLKLVLDELRRLDVGPNP...,MQNLKIQEEVNSLMRLGQHFDDQLKLASVELGDFSDDDLALLDKCA...,394,Q9W0S8,1
...,...,...,...,...,...,...,...,...,...,...,...
431133,480480,P9WNU1,P9WNT5,DPO3B_MYCTU,DNAE2_MYCTU,MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,MFDILWNVGWSNGPPSWAEMERVLNGKPRHAGVPAFDADGDVPRSR...,MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,1500,P9WNU1,1
431134,480481,P9WNU1,P9WNT7,DPO3B_MYCTU,DPO3A_MYCTU,MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,MSGSSAGSSFVHLHNHTEYSMLDGAAKITPMLAEVERLGMPAVGMT...,MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,1586,P9WNU1,1
431135,480482,P9WNU1,P9WNT3,DPO3B_MYCTU,DPO41_MYCTU,MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,MESRWVLHLDMDAFFASVEQLTRPTLRGRPVLVGGLGGRGVVAGAS...,MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,870,P9WNU1,1
431136,480483,P05648,P45707,DNAA_BACSU,SIRA_BACSU,MENILDLWNQALAQIEKKLSKPSFETWMKSTKAHSLQGDTLTITAP...,MERHYYTYLIKEEFANHYFGRESVMFELFQDYHWTSLEKQQYEMTE...,MENILDLWNQALAQIEKKLSKPSFETWMKSTKAHSLQGDTLTITAP...,594,P05648,1


In [88]:
seq1_train = df2['seq_1'].values.tolist()
seq2_train  = df2['seq_2'].values.tolist()
len(seq1_train)

431138

In [89]:
from datasets import DatasetDict, Dataset

train_dataset = Dataset.from_dict({'sequence_1': seq1_train,
                                   'sequence_2': seq2_train})
test_dataset = Dataset.from_dict({'sequence_1': seq1,
                                  'sequence_2': seq2,})

dataset_dict = DatasetDict({
    'train': train_dataset,
    'valid': test_dataset
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['sequence_1', 'sequence_2'],
        num_rows: 431138
    })
    valid: Dataset({
        features: ['sequence_1', 'sequence_2'],
        num_rows: 49348
    })
})

In [91]:
small_dataset_dict = DatasetDict({
    'train': dataset_dict['train'].select(range(10)),
    'valid': dataset_dict['valid'].select(range(10))
})
small_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['sequence_1', 'sequence_2'],
        num_rows: 10
    })
    valid: Dataset({
        features: ['sequence_1', 'sequence_2'],
        num_rows: 10
    })
})

In [92]:
small_dataset_dict.save_to_disk('/data/rozen/home/e0833634/lama/data/ppi_8000/ppi_8000_raw.hf')

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [2]:
a = [[2, 3, 4, 5], [1,2,3,4,5,6,7]]
a

[[2, 3, 4, 5], [1, 2, 3, 4, 5, 6, 7]]

In [3]:
[seq[::-1] for seq in a]

[[5, 4, 3, 2], [7, 6, 5, 4, 3, 2, 1]]

In [5]:
seed_everything(42)

global_tokenizer = None


def init_pool(tokenizer):
    global global_tokenizer
    tokenizer_path = '/data/rozen/home/e0833634/lama/protllama/batch_script/'
    tokenizer = spm.SentencePieceProcessor(model_file=tokenizer_path+"protein_8k.model")
    global_tokenizer = tokenizer


def standalone_tokenize_function(tup, max_sequence_length, extra_toks_per_seq=2):
    tokenizer_path = '/data/rozen/home/e0833634/lama/protllama/batch_script/'
    tokenizer = spm.SentencePieceProcessor(model_file=tokenizer_path+"protein_8k.model")
    global_tokenizer = tokenizer
    s1, s2 = tup
    try:
        tokens_1 = global_tokenizer.encode(s1)
        tokens_2 = global_tokenizer.encode(s2)
        return tokens_1, tokens_2
    except Exception as e:
        raise ValueError(f"Error during tokenization of string {s1} {s2}: {e}")


class TokenizeBatch(object):
    """add padding, create labels for GPT-alike training, used as collate_fn, need processed batch indices
    processed (labels + tensor) batch.
    """

    def __init__(self, tokenizer):
        self.pad_token_id = tokenizer.unk_id()

    def __call__(self, batches):
        data_tokens = [torch.tensor(token_list) for token_list in batches]
        data_tokens_padded = pad_sequence(data_tokens, batch_first=True, padding_value=self.pad_token_id)

        # Create attention masks
        attention_masks = (data_tokens_padded != self.pad_token_id).long()

        # skip label==-100 during training so that these tokens won't be used in loss calculation
        labels = data_tokens_padded.clone()
        labels[data_tokens_padded == self.pad_token_id] = -100

        return {
            'input_ids': data_tokens_padded,
            'attention_mask': attention_masks,
            'labels': labels
        }


class BatchedPPIDataset(object):
    """inspired by esm2, but instead of sorting the original sequences,
    we should really sorting based on tokenized sequences
    """

    def __init__(self, sequence_strs, tokenizer, max_sequence_length):
        self.batch_indices = None
        self.sequence_str_1 = sequence_strs['sequence_1']
        self.sequence_str_2 = sequence_strs['sequence_2']
        self.tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length
        self.tokenized_sequences = []
        self.accumulated_length = 0
        # automatically tokenize sequences upon creation of the object
        # if need manual, change it to call object.tokenize_sequence() in a separate line

    def tokenize_sequences_forward(self):
        prot_tuples = list(zip(self.sequence_str_1, self.sequence_str_2))
        tokenized_pairs  = []
        for i in prot_tuples:
            tokenized_pairs.append(standalone_tokenize_function(i, max_sequence_length=self.max_sequence_length))

        for tokens_1, tokens_2 in tokenized_pairs:
            seq_length = len(tokens_1) + len(tokens_2) + 2  # for both bos, eos tokens
            if seq_length <= self.max_sequence_length:
                forward_sequence = [self.tokenizer.bos_id()] + tokens_1 + [self.tokenizer.eos_id()] + tokens_2
                self.tokenized_sequences.append(forward_sequence)
        print(self.tokenized_sequences)

    def tokenize_sequences_backward(self):
        self.reversed_tokenized_sequences=[]
        for sequence in self.tokenized_sequences:
            eos_position = sequence.index(self.tokenizer.eos_id())
            print('backward', sequence)
            tokens_1 = sequence[1:eos_position]  # Extract tokens1 without bos and eos
            tokens_2 = sequence[eos_position+1:] # Extract tokens2 after eos
            reversed_sequence = [self.tokenizer.bos_id()] + tokens_2 + [self.tokenizer.eos_id()] + tokens_1
            self.reversed_tokenized_sequences.append(reversed_sequence)
        self.tokenized_sequences.extend(self.reversed_tokenized_sequences)

    def process_all(self):
        self.tokenize_sequences_forward()
        forward_batches = self.process_chunk(self.tokenized_sequences, self.get_batch_indices())
        offset = len(self.tokenized_sequences)
        self.tokenize_sequences_backward()
        backward_batches = self.process_chunk(self.tokenized_sequences, self.get_batch_indices(offset))
        self.tokenized_sequences = []
        combined_dataset = concatenate_datasets([forward_batches, backward_batches])
        # shuffle the datasets overall again
        shuffled_dataset = combined_dataset.shuffle()
        return shuffled_dataset

    def get_batch_indices(self, offset=0, end=None):
        if end is None:
            end=len(self.tokenized_sequences)
        sizes = [(len(tokens), i) for i, tokens in enumerate(self.tokenized_sequences[offset:end])]
        sizes = [(sz, idx + offset) for sz, idx in sizes]
        sizes.sort()
        batches = []
        buf = []
        current_buf_len = 0

        def _flush_current_buf():
            nonlocal current_buf_len, buf
            if len(buf) == 0:
                return
            batches.append(buf)
            buf = []
            current_buf_len = 0
            #print('my batches is:')
            #print(batches)

        for sz, i in sizes:
            # sz already has the length of special tokens, handled at tokenization level
            # check accumulative seq length in the buffer
            if current_buf_len + sz > self.max_sequence_length:
                _flush_current_buf()
            buf.append(i)
            current_buf_len += sz
            #print('my buffer is:')
            #print(buf)

        _flush_current_buf()
        return batches

    def process_chunk(self, tokenized_sequences, batch_indices):
        token_batch_fn = TokenizeBatch(self.tokenizer)
        print(batch_indices)
        processed_batches = [
            token_batch_fn([tokenized_sequences[i] for i in batch]) for batch
            in batch_indices]
        assert len(processed_batches) == len(batch_indices)

        # Shuffle together using a permutation
        permutation = list(torch.randperm(len(processed_batches)))
        processed_batches = [processed_batches[i] for i in permutation]

        all_attention_masks = []
        all_input_ids = []
        all_labels = []

        all_attention_masks.extend([batch['attention_mask'] for batch in processed_batches])
        all_input_ids.extend([batch['input_ids'] for batch in processed_batches])
        all_labels.extend([batch['labels'] for batch in processed_batches])

        combined_dataset = Dataset.from_dict({
            'attention_mask': all_attention_masks,
            'input_ids': all_input_ids,
            'labels': all_labels
        })
        print(combined_dataset)
        return combined_dataset


Global seed set to 42


In [70]:
text1 = ['GLTNAFIASAPAREVRYDGVITPANANYRFMGGDKGGSLTVGSHLTGSNMVTIGPMGVVVFTNNNDYTGNTFIMGGGTLQLGSNTAWGSLPN\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n',
        'MDRLDFGGGE\n']
text2 = ['MDRLDFGGGE\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFPLSPAQLGIWYAQHLDPQVPITIAQYVDLHGALDVEVLERASIDASHELGSGFLRIVERDGEPLQYV\n',
        'MDRLDFGGNGEAGSEVAPVPVSGQPVSSEQLFP\n',
        'MDRLDFGE\n']
train_dataset = Dataset.from_dict({'sequence_1': text1,
                                   'sequence_2': text2,})
train_dataset

Dataset({
    features: ['sequence_1', 'sequence_2'],
    num_rows: 4
})

In [3]:
train_dataset=load_from_disk(path+'/ppi_8000_raw.hf')
train_dataset

DatasetDict({
    train: Dataset({
        features: ['sequence_1', 'sequence_2'],
        num_rows: 431138
    })
    valid: Dataset({
        features: ['sequence_1', 'sequence_2'],
        num_rows: 49348
    })
})

In [7]:
import sentencepiece as spm
tokenizer_path = '/data/rozen/home/e0833634/lama/protllama/batch_script/'
tokenizer = spm.SentencePieceProcessor(model_file=tokenizer_path+"protein_32k.model")
dataset = BatchedPPIDataset(train_dataset['train'], tokenizer, 1024).process_all()
dataset

KeyboardInterrupt: 

In [72]:
def visualize_batched_dataset(dataset, num_batches=5):
    for batch_idx in range(min(num_batches, len(dataset))):
        batch = dataset[batch_idx]
        input_ids_batch = batch['input_ids']
        attention_mask_batch = batch['attention_mask']
        labels_batch = batch['labels']

        print(f"Batch {batch_idx + 1}")
        print("----" * 20)

        for seq_idx, input_ids in enumerate(input_ids_batch):
            print(f"Sample {seq_idx + 1} in Batch {batch_idx + 1}:")
            print("Input IDs:", input_ids)
            print("Attention Mask:", attention_mask_batch[seq_idx])
            print("Labels:", labels_batch[seq_idx])
            print("----" * 20)
# Call the function
visualize_batched_dataset(dataset)

Batch 1
--------------------------------------------------------------------------------
Sample 1 in Batch 1:
Input IDs: [1, 854, 1642, 2494, 2, 854, 1642, 653, 91, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Labels: [1, 854, 1642, 2494, 2, 854, 1642, 653, 91, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
--------------------------------------------------------------------------------
Sample 2 in Batch 1:
Input IDs: [1, 854, 1642, 653, 1279, 639, 86, 1195, 1214, 6817, 49, 141, 2076, 2, 854, 1642, 653, 1279, 639, 86, 1195, 1214, 6817, 49, 141, 2076]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Labels: [1, 854, 1642, 653, 1279, 639, 86, 1195, 1214, 6817, 49, 141, 2076, 2, 854, 1642, 653, 1279, 639, 86, 1195, 1214, 6817, 49, 141, 2076]
-------------------------------------------------